# Task 4: Obstacle avoidance

"What happens in the scenario shown in figure (10) of RiMEA (bottleneck), if obstacle avoidance is not implemented? What happens for the "chicken test" scenario?"

In [ ]:
# always import addroot first
import addroot

In [ ]:
# we will import everything else here
from src.ca import CrowdModelCellularAutomaton
from src.config import get_save_figure_function

save_current_figure = get_save_figure_function("4_obstacle_avoidance")


# set matplotlib to interactive mode -> requires ipympl to be installed
%matplotlib widget

%reload_ext autoreload
%autoreload 2

## 1. Set up scenarios.

In [ ]:
bottleneck_grid_size = (26, 63)
bottleneck_simulation_time = 100


def set_up_bottleneck(ca):
    # should have a grid size of (26, 63) 
    ca.set_area_obstacle((0, 25), (11, 25))
    ca.set_area_obstacle((14, 25), (25, 25))
    ca.set_area_obstacle((11, 26), (11, 38))
    ca.set_area_obstacle((14, 26), (14, 38))
    ca.set_area_obstacle((0, 38), (11, 38))
    ca.set_area_obstacle((14, 38), (25, 38))
    ca.set_area_pedestrian_random_n(150, (0, 0), (25, 12))
    ca.set_area_target((12, 62), (13, 62))
    ca.save_state()

In [ ]:
chicken_test_grid_size = (25, 50)
chicken_test_simulation_time = 30


def set_up_chicken_test(ca):
    # should have a grid size of (25, 50)
    ca.set_area_obstacle((5, 20), (5, 30))
    ca.set_area_obstacle((19, 20), (19, 30))
    ca.set_area_obstacle((6, 30), (18, 30))
    ca.set_cell_pedestrian((12, 10))
    ca.set_cell_target((12, 40))
    ca.save_state()

## 2. No obstacle avoidance.
 
### 2.1 Go through obstacles.

If by "without obstacle avoidance" it is meant that pedestrians can simply go through obstacles, then we have this:
 
#### 2.1.1 For the bottleneck scenario.

In [ ]:
ca1_1 = CrowdModelCellularAutomaton(bottleneck_grid_size, 
                                    constant_cost_map="no_obstacle_avoidance", 
                                    no_obstacle_avoidance=True)
set_up_bottleneck(ca1_1)
ca1_1.plot_state()
save_current_figure("bottleneck_scenario")

In [ ]:
ca1_1.simulate(start_at_saved_state=True, seconds=bottleneck_simulation_time)

In [ ]:
ca1_1.plot_simulation_end_state()
save_current_figure("bottleneck_no_avoidance")

In [ ]:
ca1_1.plot_simulation_with_time_slider()

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("bottleneck_no_avoidance_slider")

#### 2.1.2 For the chicken test scenario.

In [ ]:
ca1_2 = CrowdModelCellularAutomaton(chicken_test_grid_size, 
                                    constant_cost_map="no_obstacle_avoidance", 
                                    no_obstacle_avoidance=True)
set_up_chicken_test(ca1_2)
ca1_2.plot_state()
save_current_figure("chicken_test_scenario")

In [ ]:
ca1_2.simulate(start_at_saved_state=True, seconds=chicken_test_simulation_time)

In [ ]:
ca1_2.plot_simulation_end_state()
save_current_figure("chicken_test_no_avoidance")

In [ ]:
ca1_2.plot_simulation_with_time_slider()

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("chicken_test_no_avoidance_slider")

CLearly, this is just broken. The obstacle cells turn into pedestrian cells and are removed.
 
### 2.2 No obstacle avoidance strategy.

If by "without obstacle avoidance" it is meant that pedestrians do not have a strategy to get around obstacles (simply using the euclidean distance to the target), then we have this:
 
#### 2.2.1 For the bottleneck scenario.

In [ ]:
ca2_1 = CrowdModelCellularAutomaton(bottleneck_grid_size, constant_cost_map="min_euclidean_distance")
set_up_bottleneck(ca2_1)
ca2_1.simulate(start_at_saved_state=True, seconds=bottleneck_simulation_time)

In [ ]:
ca2_1.plot_simulation_end_state()
save_current_figure("bottleneck_euclidean_distance")

In [ ]:
ca2_1.plot_simulation_with_time_slider()

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("bottleneck_euclidean_distance_slider")

Clearly, the pedestrians behave in a sub-optimal and strange fashion. However, they do end up all reaching the target, and the test criterion of scenario 12 is even passed (only room 1 has a congestion, not room 2).

#### 2.2.2 For the chicken test scenario.

In [ ]:
ca2_2 = CrowdModelCellularAutomaton(chicken_test_grid_size, constant_cost_map="min_euclidean_distance")
set_up_chicken_test(ca2_2)
ca2_2.simulate(start_at_saved_state=True, seconds=chicken_test_simulation_time)

In [ ]:
ca2_2.plot_simulation_end_state()
save_current_figure("chicken_test_euclidean_distance")

In [ ]:
ca2_2.plot_simulation_with_time_slider()

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("chicken_test_euclidean_distance_slider")

In the chicken test scenario the pedestrian fails to reach the target without a strategy to avoid the large C-shaped obstacle in front of it.

## 3. With obstacle avoidance.

With the Dijkstra algorithm we can compute the shortest path from any cell to the target. Then we can save this as a cost value, and simply let the pedestrian choose locally which neighboring cell has the smallest cost.

### 3.1 For the bottleneck scenario.

In [ ]:
ca3_1 = CrowdModelCellularAutomaton(bottleneck_grid_size, constant_cost_map="shortest_path")
set_up_bottleneck(ca3_1)
ca3_1.simulate(start_at_saved_state=True, seconds=bottleneck_simulation_time)

In [ ]:
ca3_1.plot_simulation_end_state()
save_current_figure("bottleneck_shortest_path")

In [ ]:
ca3_1.plot_simulation_with_time_slider()

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("bottleneck_shortest_path_slider")

Now the pedestrians behave in a much more natural way, joining up in the middle of the room. And the test criterion of scenario 12 is still passed (only room 1 has a congestion, not room 2). We can see why this is the case by comparing the cost maps. Before, we set the cost to the minimum distance to a target:

In [ ]:
ca2_1.plot_constant_cost_map(fig_size=(7, 3))
save_current_figure("bottleneck_euclidean_distance_cost")

And now the cost is the distance of the shortest actual path that leads to the target:

In [ ]:
ca3_1.plot_constant_cost_map(fig_size=(7, 3))
save_current_figure("bottleneck_shortest_path_cost")

### 3.2 For the chicken test scenario.

In [ ]:
ca3_2 = CrowdModelCellularAutomaton(chicken_test_grid_size, constant_cost_map="shortest_path")
set_up_chicken_test(ca3_2)
ca3_2.simulate(start_at_saved_state=True, seconds=chicken_test_simulation_time)

In [ ]:
ca3_2.plot_simulation_end_state()
save_current_figure("chicken_test_shortest_path")

In [ ]:
ca3_2.plot_simulation_with_time_slider()

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("chicken_test_shortest_path_slider")

And now the pedestrian is able to avoid the obstacle in the chicken test! Again, we can compare the cost maps to see how it works:. Before, with the euclidean distance:

In [ ]:
ca2_2.plot_constant_cost_map(fig_size=(7, 3))
save_current_figure("chicken_test_euclidean_distance_cost")

Now with the shortest path distance:

In [ ]:
ca3_2.plot_constant_cost_map(fig_size=(7, 3))
save_current_figure("chicken_test_shortest_path_cost")

Done!